In [1]:
import sys
import mosek
import numpy as np
from itertools import permutations
from math import sqrt,log,factorial,floor
import random 
import time
#from sympy import Matrix
import seaborn as sns
import pandas as pd

In [2]:
def prefer(loci,locj): 
    # returns 1/2 if both candidates are preferred equally.
    # returns 1 if c_i is preferred than c_j and 0 otherwise.
    if loci==locj:
        val=1/2
    else:
        val=int(loci<locj)
    return val

def prefermatrix(rank):
    # S(t): returns the preference matrix for a given ordering.
    loc=[rank.index(i) for i in candidate]
    a=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i!=j:
                a[i,j]= prefer(loc[i],loc[j])
    return a

def FrobeniusNorm(a):
    norm=0
    b=a.shape[0]
    for i in range(1,b):
        for j in range(i):
            norm+=2*(a[i,j])**2
    return sqrt(norm)

def entropy(xx):
    ent=0
    negative=0
    zero=0
    for i in range(T):
        if xx[i]<0:
            negative+=1
            continue
        if xx[i]==0:
            zero+=1
            continue
        ent-=xx[i]*log(xx[i])
        
    print('the number of negative values: ',negative,'; the number of 0: ',zero)
    return ent

def matrix2vector(m):
    n=m.shape[0]
    Aj=np.zeros((int(n*(n-1)/2),1))
    index=0
    for i in range(n):
        for j in range(i):
            #f i < j:
            Aj[index,0]=m[i,j]
            index+=1
    return Aj

def assess(xx):
    outcome=np.zeros((n,n))
    for i in range(1,n):
        for j in range(i):
                outcome[i,j]=sum(theoretical[t][i,j]*xx[t] for t in range(T))
                #outcome[j,i]=1-outcome[i,j]
    #print('The original matrix is:\n',empirical)        
    #print('The output matrix is:\n',outcome)
    #print('The entropy of u is:',entropy(xx))
    #print('The Frobenius Norm of the difference is:',FrobeniusNorm(empirical-outcome))
    return FrobeniusNorm(empirical-outcome), entropy(xx)

def report(xx,u,status,runtime):
    """
    yy=xx[u:u+T]
    yy.sort(reverse=True)
    print('Sorted probabilities:\n',yy)
    """
    xxx=np.zeros((T))
    count=0
    for t in range(T):
        if xx[t] > 10**(-5):
            xxx[t]=xx[t]
            count+=1
    #print('the number of rank is:',count)
    print('the status is:',status)
    print('the runtime is:',runtime)
    print('the size of the orderings set is',T)

def msk_relent(task, t, x, y):
    v = msk_newvar(task, 1)
    c = msk_newcon(task, 1)
    task.putaij(c, v, 1.0)
    task.putaij(c, t, 1.0)
    task.putconbound(c, mosek.boundkey.fx, 0.0, 0.0)
    task.appendcone(mosek.conetype.pexp, 0.0, [y, x, v])

def msk_newvar(task, num):  # free
    v = task.getnumvar()
    task.appendvars(num)
    for i in range(num):
        task.putvarbound(v+i, mosek.boundkey.fr, -inf, inf)
    return v

def msk_newvar_fx(task, num, val):  # fixed
    v = task.getnumvar()
    task.appendvars(num)
    for i in range(num):
        task.putvarbound(v+i, mosek.boundkey.fx, val, val)
    return v

def msk_newcon(task, num):
    c = task.getnumcon()
    task.appendcons(num)
    return c

def msk_sq(task, t, x): # t >= x^2
    task.appendcone(mosek.conetype.rquad, 0.0, [msk_newvar_fx(task, 1, 0.5), t, x])

In [3]:
def mev_shortvector(empirical,theoretical):
    ### MEV ==================================================================

    env  = mosek.Env()
    task = env.Task(0,0)

    # variables -------------------------------------------------------------
    # t is the entropy of u
    eta = msk_newvar(task, T)
    u = msk_newvar(task, T)

    # constraints ------------------------------------------------------------

    # 0<u<1 
    for i in range(T):
        task.putvarbound(u+i, mosek.boundkey.ra, 0, 1)
        task.putvarbound(eta+i , mosek.boundkey.ra, -inf, 0)

    # sum(u)=1
    c = msk_newcon(task, 1)
    task.putarow(c, range(u,u+T),[1.0]*T)
    #task.putaij(c, 1, 1.0)
    task.putconbound(c, mosek.boundkey.fx, 1.0, 1.0)    

    # max entropy
    for i in range(T):
        msk_relent(task, eta+i, u+i, msk_newvar_fx(task, 1, 1.0)) #(t,x,y) t >= x * log(x/y), x,y>=0  --> -t <= -x * log x 

    # u*theoretical=empirical
    b=matrix2vector(empirical)
    A=matrix2vector(theoretical[0])
    for k in range(1,T):
        A=np.concatenate((A,matrix2vector(theoretical[k])),axis=1)

    for row in range(A.shape[0]):
        c = msk_newcon(task, 1)
        task.putarow(c, range(u, u+T),A[row,:].tolist())
        task.putconbound(c, mosek.boundkey.fx, b[row][0], b[row][0])
            
    # objective ----------------------------------------------------------------
    task.putclist(range(eta,eta+T),[1]*T)
    #task.putclist(range(task.getnumvar()),[1.0]*task.getnumvar())

    # Input the objective sense (minimize/maximize)
    task.putobjsense(mosek.objsense.minimize)
    
    # Solve the problem and print summary
    starttime = time.time()
    task.optimize()
    endtime = time.time()

    # get solution of the diagonal (equal to the SDP var)
    xx = [0.0] * task.getnumvar()
    task.getxx(mosek.soltype.itr,xx)
    #print(xx[u:u+T])
    #report(xx,u,task.getsolsta(mosek.soltype.itr),endtime-starttime)
    norm,ent=assess(xx[u:u+T])
    return [endtime-starttime, norm,ent]

In [70]:
def mev_shortvector_JLaxis0(empirical,theoretical,target_dim):
    ### MEV ==================================================================

    env  = mosek.Env()
    task = env.Task(0,0)

    # variables -------------------------------------------------------------
    # t is the entropy of u
    eta = msk_newvar(task, T)
    u = msk_newvar(task, T)

    # constraints ------------------------------------------------------------

    # 0<u<1 
    for i in range(T):
        task.putvarbound(u+i, mosek.boundkey.ra, 0, 1)
        task.putvarbound(eta+i , mosek.boundkey.ra, -inf, 0)

    # sum(u)=1
    c = msk_newcon(task, 1)
    task.putarow(c, range(u,u+T),[1.0]*T)
    #task.putaij(c, 1, 1.0)
    task.putconbound(c, mosek.boundkey.fx, 1.0, 1.0)    

    # max entropy
    for i in range(T):
        msk_relent(task, eta+i, u+i, msk_newvar_fx(task, 1, 1.0)) #(t,x,y) t >= x * log(x/y), x,y>=0  --> -t <= -x * log x 

    # =======================
    # A x = b
    # u*theoretical=empirical
    # =======================
    b_tem=matrix2vector(empirical)
    A_tem=matrix2vector(theoretical[0])
    for t in range(1,T):
        A_tem=np.concatenate((A_tem,matrix2vector(theoretical[t])),axis=1)
    print('A_tem shape:',A_tem.shape)
    tem=np.concatenate((A_tem,b_tem),axis=1) #.shape
    #k=int(log(T+1))
    #k=int(T/2)
    #R=np.random.randint(0,2,(int(n*(n-1)/2),target_dim))
    #R[R==0]=-1
    R=np.random.rand(int(n*(n-1)/2),target_dim)
    #print(R)
    R[R<1/6]=-1
    R[R>5/6]=1
    #print(R)
    R[(R>1/6) & (R<5/6)]=0
    E=tem.T.dot(R)/sqrt(target_dim)
    A=E[:T].T
    print('A shape:',A.shape)
    b=E[T].reshape((A.shape[0],1))

    for row in range(A.shape[0]):
        c = msk_newcon(task, 1)
        task.putarow(c, range(u, u+T),A[row,:].tolist())
        task.putconbound(c, mosek.boundkey.fx, b[row][0], b[row][0])
    
    # objective ----------------------------------------------------------------
    task.putclist(range(eta,eta+T),[1]*T)
    #task.putclist(range(task.getnumvar()),[1.0]*task.getnumvar())

    # Input the objective sense (minimize/maximize)
    task.putobjsense(mosek.objsense.minimize)
    
    # Solve the problem and print summary
    starttime = time.time()
    task.optimize()
    endtime = time.time()

    # get solution of the diagonal (equal to the SDP var)
    xx = [0.0] * task.getnumvar()
    task.getxx(mosek.soltype.itr,xx)
    #print(xx[u:u+T])
    #report(xx,u,task.getsolsta(mosek.soltype.itr),endtime-starttime)
    norm,ent=assess(xx[u:u+T])
    return [endtime-starttime, norm,ent]

In [68]:
df=pd.DataFrame(columns=['runtime','RP_error','entropy','met','the number of candidates'],index=[*range(100)])
rows=0
for n in range(5,9):
    candidate=['c'+str(i) for i in range(1,1+n)]
    inf=100
    
    # Get all strong orderings (permutations)
    perm=permutations(candidate)
    T=sum(1 for ignore in perm)
    # distRD[t] Gives the proportion of people who vote for ordering t.

    for r in range(1,9):
    #    start=1
    #    distRD=[]
    #    for i in range(T-1):
    #        distRD+=[random.uniform(0,start)]
    #        start=1-sum(distRD)
    #    distRD+=[start]
        
        #distRD=[r/T/sum(range(1,r+1)) for i in range(1,r+1)]*int(T/r)
        a=random.uniform(0.1,0.8)
        b=random.uniform(0.1,1-a)
        c=1-a-b
        distRD=[a/T*3,b/T*3,c/T*3]*int(T/3)
        
        # empirical preference matrix: \hat(S)(V) 
        # theoretical preference matrices for each strong ordering.S(t)
        empirical=np.zeros((n,n))
        theoretical={}
        index=0
        for i in permutations(candidate):
            empirical+=prefermatrix(i)*distRD[index]
            theoretical[index]=prefermatrix(i)
            index+=1
            
        df.loc[rows,:]=mev_shortvector_JLaxis0(empirical,theoretical,int(1.5*log(n)))+['1.5 * log n']+[n] # original size: (n-1)*n/2
        df.loc[rows+1,:]=mev_shortvector_JLaxis0(empirical,theoretical,int(1.5*log(n)**3))+['1.5 * log n ** 3']+[n] # original size: (n-1)*n/2
        #df.loc[rows+1,:]=mev_shortvector_JLaxis1(empirical,theoretical,int(T/10))+['cone_JL_axis1']+[n] # original size: T
        df.loc[rows+2,:]=mev_shortvector(empirical,theoretical)+['n(n-1)/2']+[n] 
        rows+=3
df=df.astype({'runtime': 'float32','RP_error': 'float32','entropy': 'float32','the number of candidates': 'float32'})

A_tem shape: (10, 120)
A shape: (2, 120)
the number of negative values:  0 ; the number of 0:  0
A_tem shape: (10, 120)
A shape: (6, 120)
the number of negative values:  0 ; the number of 0:  0
the number of negative values:  0 ; the number of 0:  0
A_tem shape: (10, 120)
A shape: (2, 120)
the number of negative values:  0 ; the number of 0:  0
A_tem shape: (10, 120)
A shape: (6, 120)
the number of negative values:  0 ; the number of 0:  0
the number of negative values:  0 ; the number of 0:  0
A_tem shape: (10, 120)
A shape: (2, 120)
the number of negative values:  0 ; the number of 0:  0
A_tem shape: (10, 120)
A shape: (6, 120)
the number of negative values:  0 ; the number of 0:  0
the number of negative values:  0 ; the number of 0:  0
A_tem shape: (10, 120)
A shape: (2, 120)
the number of negative values:  0 ; the number of 0:  0
A_tem shape: (10, 120)
A shape: (6, 120)
the number of negative values:  0 ; the number of 0:  0
the number of negative values:  0 ; the number of 0:  0


In [69]:
df.to_csv('MEV_JL_new.csv', index=False)

In [41]:
df=df.astype({'runtime': 'float32','RP_error': 'float32','entropy': 'float32','the number of candidates': 'float32'})

# End

In [5]:
def mev_shortvector_JLaxis0_old(empirical,theoretical,target_dim):
    ### MEV ==================================================================

    env  = mosek.Env()
    task = env.Task(0,0)

    # variables -------------------------------------------------------------
    # t is the entropy of u
    eta = msk_newvar(task, T)
    u = msk_newvar(task, T)

    # constraints ------------------------------------------------------------

    # 0<u<1 
    for i in range(T):
        task.putvarbound(u+i, mosek.boundkey.ra, 0, 1)
        task.putvarbound(eta+i , mosek.boundkey.ra, -inf, 0)

    # sum(u)=1
    c = msk_newcon(task, 1)
    task.putarow(c, range(u,u+T),[1.0]*T)
    #task.putaij(c, 1, 1.0)
    task.putconbound(c, mosek.boundkey.fx, 1.0, 1.0)    

    # max entropy
    for i in range(T):
        msk_relent(task, eta+i, u+i, msk_newvar_fx(task, 1, 1.0)) #(t,x,y) t >= x * log(x/y), x,y>=0  --> -t <= -x * log x 

    # =======================
    # A x = b
    # u*theoretical=empirical
    # =======================
    b_tem=matrix2vector(empirical)
    A_tem=matrix2vector(theoretical[0])
    for t in range(1,T):
        A_tem=np.concatenate((A_tem,matrix2vector(theoretical[t])),axis=1)
    print('A_tem shape:',A_tem.shape)
    tem=np.concatenate((A_tem,b_tem),axis=1) #.shape
    #k=int(log(T+1))
    #k=int(T/2)
    R=np.random.randint(0,2,(int(n*(n-1)/2),target_dim))
    #R[R==0]=-1
    E=tem.T.dot(R)#/sqrt(target_dim)
    A=E[:T].T
    print('A shape:',A.shape)
    b=E[T].reshape((A.shape[0],1))

    for row in range(A.shape[0]):
        c = msk_newcon(task, 1)
        task.putarow(c, range(u, u+T),A[row,:].tolist())
        task.putconbound(c, mosek.boundkey.fx, b[row][0], b[row][0])
    
    # objective ----------------------------------------------------------------
    task.putclist(range(eta,eta+T),[1]*T)
    #task.putclist(range(task.getnumvar()),[1.0]*task.getnumvar())

    # Input the objective sense (minimize/maximize)
    task.putobjsense(mosek.objsense.minimize)
    
    # Solve the problem and print summary
    starttime = time.time()
    task.optimize()
    endtime = time.time()

    # get solution of the diagonal (equal to the SDP var)
    xx = [0.0] * task.getnumvar()
    task.getxx(mosek.soltype.itr,xx)
    #print(xx[u:u+T])
    #report(xx,u,task.getsolsta(mosek.soltype.itr),endtime-starttime)
    norm,ent=assess(xx[u:u+T])
    return [endtime-starttime, norm,ent]

In [7]:
def mev_shortvector_JLaxis1(empirical,theoretical,target_dim): # usually infeasible
    ### MEV ==================================================================

    env  = mosek.Env()
    task = env.Task(0,0)

    # variables -------------------------------------------------------------
    # t is the entropy of u
    eta = msk_newvar(task, T)
    u = msk_newvar(task, T)

    # constraints ------------------------------------------------------------

    # 0<u<1 
    for i in range(T):
        task.putvarbound(u+i, mosek.boundkey.ra, 0, 1)
        task.putvarbound(eta+i , mosek.boundkey.ra, -inf, 0)

    # sum(u)=1
    c = msk_newcon(task, 1)
    task.putarow(c, range(u,u+T),[1.0]*T)
    #task.putaij(c, 1, 1.0)
    task.putconbound(c, mosek.boundkey.fx, 1.0, 1.0)    

    # max entropy
    for i in range(T):
        msk_relent(task, eta+i, u+i, msk_newvar_fx(task, 1, 1.0)) #(t,x,y) t >= x * log(x/y), x,y>=0  --> -t <= -x * log x 

    # =======================
    # A x = b
    # u*theoretical=empirical
    # =======================
    b_tem=matrix2vector(empirical)
    A_tem=matrix2vector(theoretical[0])
    for t in range(1,T):
        A_tem=np.concatenate((A_tem,matrix2vector(theoretical[t])),axis=1)
    print('A_tem shape:',A_tem.shape)
    tem=np.concatenate((A_tem,b_tem),axis=1) #.shape (n*(n-1)/2, T+1)
    #k=int(log(T+1))
    #k=int(T/2)
    R=np.random.randint(0,2,(T+1,target_dim))
    #R[R==0]=-1
    E=tem.dot(R)/sqrt(target_dim) # shape (n*(n-1)/2, target_dim)
    E=E.T
    A=E[:target_dim-1].T #E[:target_dim] 
    print('A shape:',A.shape)
    b=E[target_dim-1].reshape((A.shape[0],1))

    for row in range(A.shape[0]):
        c = msk_newcon(task, 1)
        task.putarow(c, range(u, u+target_dim-1),A[row,:].tolist())
        task.putconbound(c, mosek.boundkey.fx, b[row][0], b[row][0])
    
    # objective ----------------------------------------------------------------
    task.putclist(range(eta,eta+T),[1]*T)
    #task.putclist(range(task.getnumvar()),[1.0]*task.getnumvar())

    # Input the objective sense (minimize/maximize)
    task.putobjsense(mosek.objsense.minimize)
    
    # Solve the problem and print summary
    starttime = time.time()
    task.optimize()
    endtime = time.time()

    # get solution of the diagonal (equal to the SDP var)
    xx = [0.0] * task.getnumvar()
    task.getxx(mosek.soltype.itr,xx)
    #print(xx[u:u+T])
    #report(xx,u,task.getsolsta(mosek.soltype.itr),endtime-starttime)
    norm,ent=assess(xx[u:u+T])
    return [endtime-starttime, norm,ent]